# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_CNN_BiLSTM_Linear import ABSA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error()

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_MAMS_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
1,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
2,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
3,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/MAMS/models/bert_pre_trained_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/MAMS/stats/bert_pre_trained_dropout_cnn_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.3631865978240967
Epoch: 0/2, Batch: 223/2237, Loss: 0.10931700468063354
Epoch: 0/2, Batch: 446/2237, Loss: 0.07141556590795517
Epoch: 0/2, Batch: 669/2237, Loss: 0.08653442561626434
Epoch: 0/2, Batch: 892/2237, Loss: 0.04957825690507889
Epoch: 0/2, Batch: 1115/2237, Loss: 0.0416187085211277
Epoch: 0/2, Batch: 1338/2237, Loss: 0.03671260550618172
Epoch: 0/2, Batch: 1561/2237, Loss: 0.03618067502975464
Epoch: 0/2, Batch: 1784/2237, Loss: 0.03889615461230278
Epoch: 0/2, Batch: 2007/2237, Loss: 0.038617756217718124
Epoch: 0/2, Batch: 2230/2237, Loss: 0.021549008786678314
Epoch: 1/2, Batch: 0/2237, Loss: 0.05912388116121292
Epoch: 1/2, Batch: 223/2237, Loss: 0.033196769654750824
Epoch: 1/2, Batch: 446/2237, Loss: 0.036182940006256104
Epoch: 1/2, Batch: 669/2237, Loss: 0.04589039459824562
Epoch: 1/2, Batch: 892/2237, Loss: 0.03612091392278671
Epoch: 1/2, Batch: 1115/2237, Loss: 0.0384889580309391
Epoch: 1/2, Batch: 1338/2237, Loss: 0.04909728094935

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.3882732391357422
Epoch: 0/2, Batch: 223/2237, Loss: 0.06157192960381508
Epoch: 0/2, Batch: 446/2237, Loss: 0.043841928243637085
Epoch: 0/2, Batch: 669/2237, Loss: 0.03719068318605423
Epoch: 0/2, Batch: 892/2237, Loss: 0.03641417250037193
Epoch: 0/2, Batch: 1115/2237, Loss: 0.029159951955080032
Epoch: 0/2, Batch: 1338/2237, Loss: 0.042774736881256104
Epoch: 0/2, Batch: 1561/2237, Loss: 0.04848344996571541
Epoch: 0/2, Batch: 1784/2237, Loss: 0.02744126319885254
Epoch: 0/2, Batch: 2007/2237, Loss: 0.02402782253921032
Epoch: 0/2, Batch: 2230/2237, Loss: 0.027957776561379433
Epoch: 1/2, Batch: 0/2237, Loss: 0.025321662425994873
Epoch: 1/2, Batch: 223/2237, Loss: 0.048340052366256714
Epoch: 1/2, Batch: 446/2237, Loss: 0.0188961923122406
Epoch: 1/2, Batch: 669/2237, Loss: 0.04096396639943123
Epoch: 1/2, Batch: 892/2237, Loss: 0.03271034359931946
Epoch: 1/2, Batch: 1115/2237, Loss: 0.0288045946508646
Epoch: 1/2, Batch: 1338/2237, Loss: 0.036006011068

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 4/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.4361921548843384
Epoch: 0/2, Batch: 223/2237, Loss: 0.08529439568519592
Epoch: 0/2, Batch: 446/2237, Loss: 0.05542869120836258
Epoch: 0/2, Batch: 669/2237, Loss: 0.053058311343193054
Epoch: 0/2, Batch: 892/2237, Loss: 0.04335419461131096
Epoch: 0/2, Batch: 1115/2237, Loss: 0.03710734844207764
Epoch: 0/2, Batch: 1338/2237, Loss: 0.0324615016579628
Epoch: 0/2, Batch: 1561/2237, Loss: 0.055429600179195404
Epoch: 0/2, Batch: 1784/2237, Loss: 0.03513846546411514
Epoch: 0/2, Batch: 2007/2237, Loss: 0.02990262396633625
Epoch: 0/2, Batch: 2230/2237, Loss: 0.02595461905002594
Epoch: 1/2, Batch: 0/2237, Loss: 0.036542199552059174
Epoch: 1/2, Batch: 223/2237, Loss: 0.022168118506669998
Epoch: 1/2, Batch: 446/2237, Loss: 0.02820616029202938
Epoch: 1/2, Batch: 669/2237, Loss: 0.043523967266082764
Epoch: 1/2, Batch: 892/2237, Loss: 0.029882553964853287
Epoch: 1/2, Batch: 1115/2237, Loss: 0.029850227758288383
Epoch: 1/2, Batch: 1338/2237, Loss: 0.0237157586

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.990441,0.990441,0.442078,0.990441,0.253916,0.990441,0.256493,2099.871010
1,0.991296,0.991296,0.517035,0.991296,0.323365,0.991296,0.354625,2027.496546
2,0.991084,0.991084,0.503016,0.991084,0.281170,0.991084,0.301803,1689.650854
3,0.991426,0.991426,0.474129,0.991426,0.331309,0.991426,0.353523,1585.099316
4,0.991547,0.991547,0.722087,0.991547,0.332457,0.991547,0.347023,1582.259509
5,0.991180,0.991180,0.470892,0.991180,0.324101,0.991180,0.355393,1582.108990
6,0.991232,0.991232,0.464236,0.991232,0.305426,0.991232,0.332749,1582.286159
7,0.991531,0.991531,0.428022,0.991531,0.333354,0.991531,0.342430,1582.126265
8,0.991049,0.991049,0.501598,0.991049,0.305600,0.991049,0.322350,1581.916067
9,0.991440,0.991440,0.597788,0.991440,0.323723,0.991440,0.339569,1657.568510


In [20]:
results.to_csv(STATS_OUTPUT)